# 📈 Sequence Models - LSTM, GRU, CNN

## Obiettivo

Questo notebook implementa **modelli sequenziali** per predire il burnout da finestre temporali di 7 giorni. A differenza dell'MLP (che usa features aggregate), qui il modello vede la **sequenza giorno per giorno**.

### Perché Modelli Sequenziali?
Il burnout si sviluppa nel tempo. Una singola giornata non cattura il trend:
- Declino progressivo della qualità del sonno
- Accumulo di stress settimana dopo settimana
- Pattern ciclici lavoro-recupero

### Architetture Implementate

#### 1. LSTM (Long Short-Term Memory)
- Reti ricorrenti con "memoria" a lungo termine
- Gates (forget, input, output) controllano il flusso di informazione
- Best per: pattern che si estendono su più giorni

#### 2. GRU (Gated Recurrent Unit)
- Versione semplificata dell'LSTM
- Meno parametri, spesso performance simile
- Best per: dataset più piccoli o training più veloce

#### 3. CNN 1D (Convolutional Neural Network)
- Filtri convoluzionali su sequenze temporali
- Cattura pattern locali (2-3 giorni consecutivi)
- Best per: pattern locali ripetuti

### Input
- `data/processed/daily_with_burnout.parquet` (dati giornalieri con burnout label)

### Output
- `models/saved/lstm_classifier.pt`, `gru_classifier.pt`, `cnn1d_classifier.pt`

In [ ]:
# =============================================================================
# IMPORT E CONFIGURAZIONE
# =============================================================================

from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

# Device selection
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Paths
DATA_DIR = Path('../data/processed')
MODEL_DIR = Path('../models/saved')
MODEL_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# =============================================================================
# CARICAMENTO DATI GIORNALIERI
# =============================================================================
# Carichiamo i dati giornalieri con il burnout_level già calcolato
# (generato da scripts/create_burnout_labels.py)

daily_path = DATA_DIR / 'daily_with_burnout.parquet'
daily = pd.read_parquet(daily_path)

# Assicuriamoci che i dati siano ordinati per utente e data
daily['date'] = pd.to_datetime(daily['date'])
daily = daily.sort_values(['user_id', 'date'])

# IMPORTANTE: Converti work_pressure da stringa a numerico
# Il dataset originale ha valori "low", "medium", "high"
if daily['work_pressure'].dtype == object:
    pressure_map = {'low': 0, 'medium': 1, 'high': 2}
    daily['work_pressure'] = daily['work_pressure'].map(pressure_map).fillna(1).astype(np.float32)

# Features per il modello sequenziale (15 metriche giornaliere)
feature_cols = [
    'sleep_hours', 'sleep_quality',      # Sonno
    'work_hours', 'meetings_count',      # Lavoro
    'tasks_completed', 'exercise_minutes', 'steps_count',  # Attività
    'caffeine_mg', 'alcohol_units', 'screen_time_hours',  # Consumo
    'stress_level', 'mood_score', 'energy_level', 'focus_score',  # Psicologici
    'work_pressure'  # Ambiente (ora numerico)
]

# Dimensione finestra: 7 giorni (una settimana lavorativa)
window = 7
print(f"Features: {len(feature_cols)}, Window: {window} days")

In [ ]:
# =============================================================================
# CREAZIONE SEQUENZE (SLIDING WINDOW)
# =============================================================================
# Per ogni utente, creiamo finestre scorrevoli di 7 giorni.
# La label è il burnout dell'ULTIMO giorno della finestra.
#
# Esempio per un utente con 10 giorni:
#   Seq 1: giorni 1-7  → label = giorno 7
#   Seq 2: giorni 2-8  → label = giorno 8
#   Seq 3: giorni 3-9  → label = giorno 9
#   Seq 4: giorni 4-10 → label = giorno 10

def build_sequences(df, features, window):
    """
    Crea sequenze sliding window dai dati giornalieri.
    
    Args:
        df: DataFrame con dati giornalieri
        features: lista di colonne feature
        window: dimensione finestra in giorni
    
    Returns:
        X: array (N_seq, window, N_features)
        y: array (N_seq,) con labels
    """
    sequences, labels = [], []
    
    for uid, group in df.groupby('user_id'):
        feats = group[features].to_numpy(dtype=np.float32)
        labs = group['burnout_level'].to_numpy(dtype=np.int64)
        
        # Skip utenti con meno di `window` giorni
        if len(group) < window:
            continue
        
        # Sliding window
        for idx in range(window, len(group) + 1):
            seq = feats[idx - window: idx]  # 7 giorni di features
            label = labs[idx - 1]            # Burnout dell'ultimo giorno
            
            # Skip se ci sono NaN
            if np.isnan(seq).any():
                continue
            
            sequences.append(seq)
            labels.append(label)
    
    return np.stack(sequences), np.array(labels)

# Costruiamo le sequenze
seq_X, seq_y = build_sequences(daily, feature_cols, window)
print(f"Total sequences: {len(seq_X):,}")
print(f"Shape: {seq_X.shape} = (sequences, days, features)")

In [ ]:
# =============================================================================
# TRAIN/VAL SPLIT E DATALOADER
# =============================================================================

# Split stratificato per mantenere proporzione classi
X_train, X_val, y_train, y_val = train_test_split(
    seq_X, seq_y, test_size=0.2, stratify=seq_y, random_state=42
)

# Wrapping in TensorDataset
train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
val_ds = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))

# DataLoader con batching
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)

# Dimensioni per i modelli
seq_len = X_train.shape[1]   # 7
input_dim = X_train.shape[2]  # 15
num_classes = len(np.unique(seq_y))  # 3
print(f"seq_len={seq_len}, input_dim={input_dim}, num_classes={num_classes}")

In [ ]:
# =============================================================================
# ARCHITETTURE DEI MODELLI
# =============================================================================

class SequenceNet(nn.Module):
    """
    Rete ricorrente (LSTM o GRU) per classificazione sequenze.
    
    Architettura:
        Input (batch, 7, 15) 
            → LSTM/GRU 2 layers (hidden=128)
            → Prendi ultimo hidden state
            → LayerNorm → ReLU → Dropout → Linear(3)
    """
    def __init__(self, input_dim, hidden_dim=128, cell='lstm'):
        super().__init__()
        # Selezione tipo di cella ricorrente
        rnn_cls = nn.LSTM if cell == 'lstm' else nn.GRU
        
        # 2 layer RNN con dropout tra i layer
        self.rnn = rnn_cls(
            input_dim, hidden_dim, 
            batch_first=True,  # Input: (batch, seq, features)
            num_layers=2, 
            dropout=0.2
        )
        
        # Classification head
        self.head = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        out, _ = self.rnn(x)       # out: (batch, seq_len, hidden)
        last = out[:, -1, :]       # Prendi ultimo timestep
        return self.head(last)


class CNN1D(nn.Module):
    """
    CNN 1D per classificazione sequenze.
    
    Architettura:
        Input (batch, 7, 15) → transpose → (batch, 15, 7)
            → Conv1d(64, kernel=3) → ReLU → BatchNorm
            → Conv1d(128, kernel=3) → ReLU → GlobalAvgPool
            → Dropout → Linear(3)
    
    I filtri convoluzionali catturano pattern locali (2-3 giorni).
    """
    def __init__(self, input_dim, seq_len):
        super().__init__()
        self.conv = nn.Sequential(
            # Conv1d: input_dim canali → 64 canali
            nn.Conv1d(input_dim, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            # 64 → 128 canali
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            # Global average pooling: (batch, 128, seq) → (batch, 128, 1)
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # x: (batch, seq_len, features)
        # Conv1d vuole: (batch, channels, seq_len)
        x = x.transpose(1, 2)
        feats = self.conv(x)
        return self.fc(feats)

In [ ]:
# =============================================================================
# FUNZIONE DI TRAINING
# =============================================================================
# Funzione generica per allenare qualsiasi modello sequenziale

def train_model(model, train_loader, val_loader, epochs=40, lr=1e-3, name='model'):
    """
    Training loop standard per modelli sequenziali.
    
    Args:
        model: PyTorch model
        train_loader, val_loader: DataLoaders
        epochs: numero di epoche
        lr: learning rate
        name: nome per salvare il checkpoint
    
    Returns:
        history: dict con train/val loss per epoca
    """
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    best_val = float('inf')
    history = {'train': [], 'val': []}
    
    for epoch in range(1, epochs + 1):
        # === Training ===
        model.train()
        tr_losses = []
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            tr_losses.append(loss.item())
        
        # === Validation ===
        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                val_losses.append(criterion(model(xb), yb).item())
        
        train_loss = np.mean(tr_losses)
        val_loss = np.mean(val_losses)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        
        # Save best model
        if val_loss < best_val:
            best_val = val_loss
            torch.save({
                'state_dict': model.state_dict(), 
                'feature_cols': feature_cols
            }, MODEL_DIR / f'{name}.pt')
        
        if epoch % 5 == 0:
            print(f"[{name}] Epoch {epoch}: train={train_loss:.4f}, val={val_loss:.4f}")
    
    return history

In [ ]:
# =============================================================================
# TRAINING DEI 3 MODELLI
# =============================================================================
# Alleniamo LSTM, GRU e CNN1D per confrontare le architetture

print("Training LSTM...")
hist_lstm = train_model(
    SequenceNet(input_dim, cell='lstm'), 
    train_loader, val_loader, 
    name='lstm_classifier'
)

print("\nTraining GRU...")
hist_gru = train_model(
    SequenceNet(input_dim, cell='gru'), 
    train_loader, val_loader, 
    name='gru_classifier'
)

print("\nTraining CNN1D...")
hist_cnn = train_model(
    CNN1D(input_dim, seq_len), 
    train_loader, val_loader, 
    name='cnn1d_classifier'
)

In [ ]:
# =============================================================================
# LEARNING CURVES
# =============================================================================
# Confronto visivo dell'andamento del training per i 3 modelli

def plot_history(history, title):
    """Plotta train/val loss."""
    plt.figure(figsize=(7, 4))
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'], label='Validation')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()

plot_history(hist_lstm, 'LSTM Training Curves')
plot_history(hist_gru, 'GRU Training Curves')
plot_history(hist_cnn, 'CNN1D Training Curves')

In [ ]:
# =============================================================================
# VALUTAZIONE FINALE
# =============================================================================
# Carichiamo i best models e confrontiamo le metriche

def evaluate_model(model_path):
    """Carica un modello e calcola predizioni sul validation set."""
    payload = torch.load(model_path, map_location=DEVICE, weights_only=False)
    state = payload['state_dict']
    name = model_path.stem
    
    # Ricostruisci l'architettura corretta
    if 'lstm' in name:
        model = SequenceNet(input_dim, cell='lstm')
    elif 'gru' in name:
        model = SequenceNet(input_dim, cell='gru')
    else:
        model = CNN1D(input_dim, seq_len)
    
    model.load_state_dict(state)
    model = model.to(DEVICE)
    model.eval()
    
    with torch.no_grad():
        xb = torch.from_numpy(X_val).to(DEVICE)
        logits = model(xb)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
    
    return preds

# Valutazione di tutti i modelli
print("=== SEQUENCE MODELS COMPARISON ===\n")
for model_name in ['lstm_classifier.pt', 'gru_classifier.pt', 'cnn1d_classifier.pt']:
    preds = evaluate_model(MODEL_DIR / model_name)
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro')
    print(f"{model_name:25s} Accuracy: {acc:.4f}, F1 Macro: {f1:.4f}")